# Decoder Toolkit Demo

This notebook demonstrates basic linear code encoding and decoding methods.
While these are not the primary focus of the project, they provide essential infrastructure 
for more advanced decoders such as those based on the **Key Equation** and the **Sudan-Guruswami** algorithm.

## Contents
1. Setup and Generator Matrix
2. Encoding Examples
3. Nearest Neighbor Decoder
4. Standard Array Decoder
5. Syndrome Decoder
6. Summary and Limitations

## 1. Setup and Generator Matrix

We use a binary linear $[7,4]$ code over $\mathbb{F}_2$ with the following generator matrix:

$$
G = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1
\end{bmatrix}
$$

In [1]:
# Imports
import numpy as np
import sys
import os
sys.path.append(os.path.abspath("../src"))
import decoders
from utils import encode

In [2]:
# Define the generator matrix for the [7,4] binary code
G = [
    [1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1]
]

## 2. Encoding Examples

We now demonstrate how to encode messages using the generator matrix \( G \) defined above.

Each message is a length-4 binary vector over \( \mathbb{F}_2 \), and encoding is performed via the matrix product:

\[
\mathbf{c} = \mathbf{u} \cdot G \mod 2
\]

This produces a codeword of length 7. We'll show a few examples below.

In [3]:
messages = [
    [0, 0, 0, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [1, 1, 1, 1]
]

print("Encoding with encode(message, G, p):")
for m in messages:
    codeword = encode(m, G, 2)
    print(f"{m} → {codeword.tolist()}")

Encoding with encode(message, G, p):
[0, 0, 0, 0] → [0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1] → [1, 0, 0, 1, 0, 0, 1]
[0, 1, 1, 0] → [0, 1, 1, 0, 1, 1, 0]
[1, 1, 1, 1] → [1, 1, 1, 1, 1, 1, 1]


## 3. Nearest Neighbor Decoder

We create an instance of the `NearestNeighborDecoder` using the generator matrix $G$ and field size $p=2$.

In [4]:
# Instantiate the decoder
nn_decoder = decoders.NearestNeighborDecoder(G, p=2)

### Code Parameters

Let's display the parameters of our $[n, k]$ code:
- $n$: codeword length
- $k$: message length
- $p$: field size

The following cell prints these parameters and the generator matrix.

In [5]:
nn_decoder.describe()

Linear [7, 4] code over 𝔽_2
Generator Matrix G:
[[1 0 0 0 1 1 0]
 [0 1 0 0 1 0 1]
 [0 0 1 0 0 1 1]
 [0 0 0 1 1 1 1]]
Parity-Check Matrix H:
[[1 1 0 1 1 0 0]
 [1 0 1 1 0 1 0]
 [0 1 1 1 0 0 1]]

First few codewords:
  [0 0 0 0] → [0 0 0 0 0 0 0]
  [0 0 0 1] → [0 0 0 1 1 1 1]
  [0 0 1 0] → [0 0 1 0 0 1 1]
  [0 0 1 1] → [0 0 1 1 1 0 0]
  [0 1 0 0] → [0 1 0 0 1 0 1]
  [0 1 0 1] → [0 1 0 1 0 1 0]
  [0 1 1 0] → [0 1 1 0 1 1 0]
  [0 1 1 1] → [0 1 1 1 0 0 1]
...


### All Codewords in the Code

Below are all possible codewords generated by our $[7,4]$ code. Each codeword corresponds to a unique message vector.  
This gives a complete picture of the code's structure.

In [6]:
nn_decoder.display_code()

Message  ->  Codeword
[0, 0, 0, 0]  ->  [0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1]  ->  [0, 0, 0, 1, 1, 1, 1]
[0, 0, 1, 0]  ->  [0, 0, 1, 0, 0, 1, 1]
[0, 0, 1, 1]  ->  [0, 0, 1, 1, 1, 0, 0]
[0, 1, 0, 0]  ->  [0, 1, 0, 0, 1, 0, 1]
[0, 1, 0, 1]  ->  [0, 1, 0, 1, 0, 1, 0]
[0, 1, 1, 0]  ->  [0, 1, 1, 0, 1, 1, 0]
[0, 1, 1, 1]  ->  [0, 1, 1, 1, 0, 0, 1]
[1, 0, 0, 0]  ->  [1, 0, 0, 0, 1, 1, 0]
[1, 0, 0, 1]  ->  [1, 0, 0, 1, 0, 0, 1]
[1, 0, 1, 0]  ->  [1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1]  ->  [1, 0, 1, 1, 0, 1, 0]
[1, 1, 0, 0]  ->  [1, 1, 0, 0, 0, 1, 1]
[1, 1, 0, 1]  ->  [1, 1, 0, 1, 1, 0, 0]
[1, 1, 1, 0]  ->  [1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1]  ->  [1, 1, 1, 1, 1, 1, 1]


### Decoding Example Received Vectors

We will decode several received vectors, including both valid codewords and vectors with single-bit errors.  
The decoder will return the closest valid message for each received vector.

In [7]:
# Messages and intentionally corrupted versions
examples = [
    ([1, 0, 1, 1], None),          # Valid message, no corruption
    ([1, 0, 1, 1], 4),             # Flip bit 4 → single-bit error
    ([0, 1, 0, 0], 2),             # Flip bit 2
    ([1, 1, 1, 1], 6),             # Flip bit 6
]

for message, flip_index in examples:
    codeword = np.dot(message, G) % 2
    received = codeword.copy()
    if flip_index is not None:
        received[flip_index] ^= 1
    decoded = nn_decoder.decode(received)
    print(f"Message: {message}")
    print(f"Encoded : {codeword.tolist()}")
    print(f"Received: {received.tolist()} (bit {flip_index} flipped)" if flip_index is not None else f"Received: {received.tolist()}")
    print(f"Decoded : {decoded}")
    print("-" * 40)

Message: [1, 0, 1, 1]
Encoded : [1, 0, 1, 1, 0, 1, 0]
Received: [1, 0, 1, 1, 0, 1, 0]
Decoded : [1 0 1 1]
----------------------------------------
Message: [1, 0, 1, 1]
Encoded : [1, 0, 1, 1, 0, 1, 0]
Received: [1, 0, 1, 1, 1, 1, 0] (bit 4 flipped)
Decoded : [1 0 1 1]
----------------------------------------
Message: [0, 1, 0, 0]
Encoded : [0, 1, 0, 0, 1, 0, 1]
Received: [0, 1, 1, 0, 1, 0, 1] (bit 2 flipped)
Decoded : [0 1 0 0]
----------------------------------------
Message: [1, 1, 1, 1]
Encoded : [1, 1, 1, 1, 1, 1, 1]
Received: [1, 1, 1, 1, 1, 1, 0] (bit 6 flipped)
Decoded : [1 1 1 1]
----------------------------------------


### Uncorrectable Example: Two-Bit Error

Now we test the decoder with a received vector that has two bits flipped.  
Since our $[7,4]$ code can only correct single-bit errors, this example demonstrates what happens when more than one error occurs.  
The decoder will still return the closest codeword, but it may not match the original message.  
We also compare the received vector to the codeword for $[1, 0, 0, 1]$ and display the Hamming distance.

In [8]:
# Uncorrectable example: flip two bits in a codeword
message = [1, 0, 1, 1]
codeword = np.dot(message, G) % 2
received = codeword.copy()
flip_indices = [2, 5]  # Flip two bits
for idx in flip_indices:
    received[idx] ^= 1

print(f"Original message: {message}")
print(f"Encoded codeword: {codeword.tolist()}")
print(f"Received vector (bits {flip_indices} flipped): {received.tolist()}")

# Compare to codeword for [1, 0, 0, 1]
other_message = [1, 0, 0, 1]
other_codeword = np.dot(other_message, G) % 2
hamming_dist = np.sum(received != other_codeword)
print(f"\nCodeword for message {other_message}: {other_codeword.tolist()}")
print(f"Hamming distance between received and codeword for {other_message}: {hamming_dist}")

decoded = nn_decoder.decode(received)
print(f"\nDecoded message: {decoded}")

Original message: [1, 0, 1, 1]
Encoded codeword: [1, 0, 1, 1, 0, 1, 0]
Received vector (bits [2, 5] flipped): [1, 0, 0, 1, 0, 0, 0]

Codeword for message [1, 0, 0, 1]: [1, 0, 0, 1, 0, 0, 1]
Hamming distance between received and codeword for [1, 0, 0, 1]: 1

Decoded message: [1 0 0 1]


## 4. Standard Array Decoder

This section is reserved for implementing a standard array decoder in the future.
For now, we focus on nearest neighbor and syndrome decoding as foundational tools.


## 5. Syndrome Decoder

The syndrome decoder uses the parity-check matrix  $H$ to compute the syndrome of a received word:

$$
\mathbf{s} = H \cdot \mathbf{r}^T \mod p
$$

For single-bit errors, the decoder matches the syndrome to a known error vector, subtracts it from the received word, and returns the corrected codeword.


In [9]:
# Create an instance of the SyndromeDecoder
synd_decoder = decoders.SyndromeDecoder(G, p=2)

# Choose a message and encode it
message = [1, 0, 1, 1]
codeword = encode(message, G, 2)

# Introduce an error (flip one bit)
received = codeword.copy()
received[4] ^= 1  # flip bit 4

# Decode
corrected = synd_decoder.decode(received)

print("Original message:", message)
print("Encoded codeword:", codeword.tolist())
print("Received (with error):", received.tolist())
print("Corrected codeword:", corrected.tolist())


Original message: [1, 0, 1, 1]
Encoded codeword: [1, 0, 1, 1, 0, 1, 0]
Received (with error): [1, 0, 1, 1, 1, 1, 0]
Corrected codeword: [1, 0, 1, 1, 0, 1, 0]


## 6. Summary and Limitations

This notebook demonstrates foundational tools for encoding and decoding linear codes,
including:

- Matrix-based encoding
- Nearest neighbor decoding
- Syndrome-based single-error correction

These tools are not the core focus of this project but provide essential groundwork for advanced decoders based on the **Key Equation** and **Sudan-Guruswami** interpolation. Future work may include expanding the syndrome decoder to correct multiple-bit errors and implementing the standard array decoder.

Limitations:
- Only binary codes (\( p = 2 \)) were used
- Syndrome decoder only handles single-bit errors
- No minimum distance estimation or advanced algebraic decoding yet

Next steps will focus on polynomial representations and key equation solvers.